In [29]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

DDoS = pd.read_csv('/content/dataset_sdn.csv')
DDoS.head()

,dt,switch,src,dst,pktcount,bytecount,dur,dur_nsec,tot_dur,flows,packetins,pktperflow,byteperflow,pktrate,Pairflow,Protocol,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label
0,11425,1,10.0.0.1,10.0.0.8,45304,48294064,100,716000000,1.010000e+11,3,1943,13535,14428310,451,0,UDP,3,143928631,3917,0,0.0,0.0,0
1,11605,1,10.0.0.1,10.0.0.8,126395,134737070,280,734000000,2.810000e+11,2,1943,13531,14424046,451,0,UDP,4,3842,3520,0,0.0,0.0,0
2,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,1943,13534,14427244,451,0,UDP,1,3795,1242,0,0.0,0.0,0
3,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,1943,13534,14427244,451,0,UDP,2,3688,1492,0,0.0,0.0,0
4,11425,1,10.0.0.2,10.0.0.8,90333,96294978,200,744000000,2.010000e+11,3,1943,13534,14427244,451,0,UDP,3,3413,3665,0,0.0,0.0,0


In [30]:
DDoS.Protocol.unique()
DDoS['Protocol'] = DDoS['Protocol'].replace('TCP', '0')
DDoS['Protocol'] = DDoS['Protocol'].replace('UDP', '1')
DDoS['Protocol'] = DDoS['Protocol'].replace('ICMP', '2')
DDoS.Protocol.unique()

array(['1', '0', '2'], dtype=object)

In [31]:
DDoS["rx_kbps"] = DDoS["rx_kbps"].fillna(DDoS["rx_kbps"].mean())

In [32]:
DDoS["tot_kbps"] = DDoS["tot_kbps"].fillna(DDoS["tot_kbps"].mean())

In [33]:
from sklearn.model_selection import train_test_split
features =DDoS[['dt', 'switch', 'pktcount', 'bytecount', 'dur', 'tot_dur',
       'flows', 'packetins', 'pktperflow', 'byteperflow', 'pktrate',
       'Pairflow','Protocol' ,'port_no', 'tx_bytes', 'rx_bytes', 'tx_kbps', 'rx_kbps',
       'tot_kbps']]
labels = DDoS['label']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [34]:
DDoS.isnull().sum()

dt             0
switch         0
src            0
dst            0
pktcount       0
bytecount      0
dur            0
dur_nsec       0
tot_dur        0
flows          0
packetins      0
pktperflow     0
byteperflow    0
pktrate        0
Pairflow       0
Protocol       0
port_no        0
tx_bytes       0
rx_bytes       0
tx_kbps        0
rx_kbps        0
tot_kbps       0
label          0
dtype: int64

In [35]:
# Feature scaling (or standardization)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [36]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(83476, 19)
(83476,)
(20869, 19)
(20869,)


In [37]:
import tensorflow
model=tensorflow.keras.models.Sequential()
#Hidden layer definitions
model.add(tensorflow.keras.layers.Dense(units=19, activation='relu', input_shape = X_train.shape[1:]))
model.add(tensorflow.keras.layers.Dense(units=50, activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.2))
model.add(tensorflow.keras.layers.Dense(units=50, activation='relu' ))
model.add(tensorflow.keras.layers.Dropout(0.2))
model.add(tensorflow.keras.layers.Dense(units=50, activation='relu' ))
#OP layer
model.add(tensorflow.keras.layers.Dense(units=2, activation='sigmoid' ))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 19)                380       
                                                                 
 dense_1 (Dense)             (None, 50)                1000      
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_3 (Dense)             (None, 50)                2550      
                                                                 
 dense_4 (Dense)             (None, 2)                 1

In [38]:
#compile
model.compile(loss='sparse_categorical_crossentropy' , optimizer='adam' , metrics=['accuracy'])

#fit
history=model.fit(x=X_train, y=y_train, validation_split=0.1, epochs=50, batch_size=16)

Epoch 1/50
4696/4696 [==============================] - 12s 2ms/step - loss: 0.1506 - accuracy: 0.9324 - val_loss: 0.0790 - val_accuracy: 0.9684
Epoch 2/50
4696/4696 [==============================] - 13s 3ms/step - loss: 0.0740 - accuracy: 0.9699 - val_loss: 0.0392 - val_accuracy: 0.9867
Epoch 3/50
4696/4696 [==============================] - 12s 3ms/step - loss: 0.0529 - accuracy: 0.9794 - val_loss: 0.0414 - val_accuracy: 0.9824
Epoch 4/50
4696/4696 [==============================] - 15s 3ms/step - loss: 0.0455 - accuracy: 0.9825 - val_loss: 0.0341 - val_accuracy: 0.9868
Epoch 5/50
4696/4696 [==============================] - 17s 4ms/step - loss: 0.0409 - accuracy: 0.9840 - val_loss: 0.0294 - val_accuracy: 0.9881
Epoch 6/50
4696/4696 [==============================] - 15s 3ms/step - loss: 0.0377 - accuracy: 0.9852 - val_loss: 0.0308 - val_accuracy: 0.9873
Epoch 7/50
4696/4696 [==============================] - 11s 2ms/step - loss: 0.0347 - accuracy: 0.9868 - val_loss: 0.0275 - val_ac

In [39]:
testloss,testaccuracy= model.evaluate(X_test, y_test)
print('test loss : ', testloss)
print('test accuracy :', testaccuracy)

653/653 [==============================] - 1s 1ms/step - loss: 0.0086 - accuracy: 0.9965
test loss :  0.00858741719275713
test accuracy : 0.9965499043464661


In [40]:
y_pred=model.predict(X_test)